In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [0]:
data_path = './drive/My Drive/ML_100/day51/'

df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

In [0]:
# 檢查row , training data中，如果某個人的缺失值太多，則捨棄

row_na = []

for i in range(len(df_train.index)):
  row_na.append( df_train.iloc[i].isnull().sum() )

row_na = np.array(row_na)
print(row_na)
result = np.where(row_na > 17)
print(result)

[ 4  5  5 11  5 13 12  5  2  5  5  4 10 11  2  5 10 18 12  7  5 11  5 11
 11 16  6 12  9  5 18  5  5  5  5 16  5  8 16 12 13 15  6 10 15 10 10  5
 10  4 14  5  5  5  7 13 16  3  9  4 11 16 17  7 11 18  4  9  2  5 17 11
 13 16 16  5 16  8 11 18  4  7 11  5 15 11  6  5 12  5 10 10 15 10 17  5
  5  4 17 15 17 11  5  9 16 16  6 20 12 12  7 10  6]
(array([ 17,  30,  65,  79, 107]),)


In [0]:
print(df_train.iloc[17]['name'] )
print(df_train.iloc[30]['name'] )
print(df_train.iloc[65]['name'] )
print(df_train.iloc[79]['name'] )
print(df_train.iloc[107]['name'] )

WROBEL BRUCE
WHALEY DAVID A
THE TRAVEL AGENCY IN THE PARK
GRAMM WENDY L
LOCKHART EUGENE E


In [0]:
print(df_train.index)

RangeIndex(start=0, stop=113, step=1)


In [0]:
df_train = df_train.drop([17 , 30 , 65 , 79 , 107] , axis = 0)

In [0]:
train_Y = df_train['poi']
name = df_test['name']
df_train = df_train.drop(['name', 'poi'] , axis=1)
df_test = df_test.drop(['name'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [0]:
# 檢查 DataFrame 空缺值的狀態
# 檢查 col
def na_check(df_data):
    # data_na is a Series
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na.values == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
na_check(df)

,Missing Ratio
loan_advances,97.163121
director_fees,88.652482
restricted_stock_deferred,87.234043
deferral_payments,72.340426
deferred_income,65.248227
long_term_incentive,53.191489
bonus,41.843972
from_messages,39.007092
from_poi_to_this_person,39.007092
from_this_person_to_poi,39.007092


In [0]:
# email address can be discarded
df = df.drop(['email_address'] , axis = 1)
print(df.shape)

(141, 19)


In [0]:
# 檢查email有沒有異常值
print('max_to_messages' , df['to_messages'].max())
print('mean_to_messages' , df['to_messages'].mean())

print('max_from_messages' , df['from_messages'].max())
print('mean_from_messages' , df['from_messages'].mean())

print('max_this_preson2poi' , df['from_this_person_to_poi'].max())
print('mean_this_preson2poi' , df['from_this_person_to_poi'].mean())

print('max_poi2this_person' , df['from_poi_to_this_person'].max())
print('mean_poi2this_person' , df['from_poi_to_this_person'].mean())

max_to_messages 15149.0
mean_to_messages 2073.860465116279
max_from_messages 14368.0
mean_from_messages 608.7906976744187
max_this_preson2poi 609.0
mean_this_preson2poi 41.23255813953488
max_poi2this_person 528.0
mean_poi2this_person 64.8953488372093


In [0]:
# 新增feature
# 以比例顯示
df['this_preson2poi_ratio'] = df['from_this_person_to_poi'].divide(df['from_messages'] , fill_value = 0.0)
df['poi2this_person_ratio'] = df['from_poi_to_this_person'].divide(df['to_messages'] , fill_value = 0.0)

df['bonus_to_salary'] = df['bonus'].divide(df['salary'] , fill_value=0.0)
df['bonus_to_total'] = df['bonus'].divide(df['total_payments'] , fill_value=0.0)

df['shared_receipt_poi_ratio'] = df['shared_receipt_with_poi'].divide(df['to_messages'] , fill_value = 0.0)

df['this_preson2poi_ratio'] = df['this_preson2poi_ratio'].fillna(0.0)
df['poi2this_person_ratio'] = df['poi2this_person_ratio'].fillna(0.0)

df['shared_receipt_poi_ratio'] = df['shared_receipt_poi_ratio'].fillna(0.0)
df['bonus_to_salary'] = df['bonus_to_salary'].fillna(0.0)
df['bonus_to_total'] = df['bonus_to_total'].fillna(0.0)

In [0]:
# 以中位數填補to_messages和from_messages
# to_messages:別人寄過來
df['to_messages'] = df['to_messages'].fillna(df['to_messages'].median())
df['from_messages'] = df['from_messages'].fillna(df['from_messages'].median())

# 以中位數填補from_this_person_to_poi和from_poi_to_this_person
df['from_this_person_to_poi'] = df['from_this_person_to_poi'].fillna(df['from_this_person_to_poi'].median())
df['from_poi_to_this_person'] = df['from_poi_to_this_person'].fillna(df['from_poi_to_this_person'].median())

# 新增feature:
df['from_this_person_to_not_poi'] = df['from_messages'] - df['from_this_person_to_poi']
df['from_not_poi_to_this_person'] = df['to_messages'] - df['from_poi_to_this_person']

In [0]:
# 檢查財務的特徵的異常值
# Salary應為百萬等級
print('max_salary' , df['salary'].max())
print('mean_salary' , df['salary'].mean())

print('max_deferral_payments' , df['deferral_payments'].max())
print('min_deferral_payments' , df['deferral_payments'].min())
print('mean_deferral_payments' , df['deferral_payments'].mean())


max_salary 26704229.0
mean_salary 562194.2947368422
max_deferral_payments 32083396.0
min_deferral_payments -102500.0
mean_deferral_payments 1642674.1538461538


In [0]:
df['salary'] = df['salary'].fillna(0.0)
df.loc[df['salary'] == 26704229.0 , 'salary'] = df['salary'].median()
print('max_salary' , df['salary'].max())

max_salary 1111258.0


In [0]:
# 小於0或超過七位數
# 參考:https://stackoverflow.com/questions/17729853/replace-value-for-a-selected-cell-in-pandas-dataframe-without-using-index
df.loc[ (df['deferral_payments'] < 0 ) | (df['deferral_payments'] > 9999999) , 'deferral_payments' ] = df['deferral_payments'].median()
df['deferral_payments'] = df['deferral_payments'].fillna(df['deferral_payments'].mean())

In [0]:
df['bonus'] = df['bonus'].fillna(df['bonus'].median())
df['deferred_income'] = df['deferred_income'].fillna(df['deferred_income'].median())
df['expenses'] = df['expenses'].fillna(df['expenses'].median())
df['total_payments'] = df['total_payments'].fillna(df['total_payments'].median())
df['total_stock_value'] = df['total_stock_value'].fillna(df['total_stock_value'].median())
df['exercised_stock_options'] = df['exercised_stock_options'].fillna(df['exercised_stock_options'].median())
df['long_term_incentive'] = df['long_term_incentive'].fillna(df['long_term_incentive'].median())
df['restricted_stock'] = df['restricted_stock'].fillna(df['restricted_stock'].median())
df['other'] = df['other'].fillna(df['other'].median())

In [0]:
df['shared_receipt_with_poi'] = df['shared_receipt_with_poi'].fillna(0.0)

In [0]:
df = df.drop(['director_fees'] , axis = 1)
df = df.drop(['loan_advances'] , axis = 1)
df = df.drop(['restricted_stock_deferred'] , axis = 1)

In [0]:
na_check(df)

,Missing Ratio


In [0]:
# log scale
df['bonus'] = np.log1p(df['bonus'])
df['exercised_stock_options'] = np.log1p(df['exercised_stock_options'])
df['total_payments'] = np.log1p(df['total_payments'])

In [0]:
na_check(df)

,Missing Ratio


In [0]:
# 將資料最大最小化
df = MinMaxScaler().fit_transform(df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

x_train , x_val , y_train , y_val = train_test_split(train_X, train_Y , test_size = 0.25 , random_state = 4)

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

n_estimators = [100, 200, 250 , 300]
max_depth = [2 , 3 , 4 , 6 ]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

# 尚未使用GridSearchCV
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=20,
                                  max_depth=6, learning_rate=0.03)


rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

In [0]:
rf.fit(x_train, y_train)
rf_pred = rf.predict_proba(x_val)[:,1]

In [0]:
auc = metrics.roc_auc_score(y_val , rf_pred)
print('auc: ' , auc)

auc:  0.861111111111111


In [0]:
# 使用GridSearchCV
grid_search = GridSearchCV(rf, param_grid, scoring="roc_auc", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(x_train, y_train)

rf_best_param = RandomForestClassifier(n_estimators = grid_result.best_params_['n_estimators'] , min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt' , max_depth = grid_result.best_params_['max_depth'] , bootstrap=True)

rf_best_param.fit(x_train , y_train)
rf_best_param_pred = rf_best_param.predict_proba(x_val)[:,1]

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   20.7s finished


In [0]:
print(grid_result.best_params_['max_depth'])
print(grid_result.best_params_['n_estimators'])

3
200


In [0]:
auc = metrics.roc_auc_score(y_val , rf_best_param_pred)
print('auc: ' , auc)

auc:  0.8333333333333334


In [0]:
store_path = './drive/My Drive/ML_100/day51/'

In [0]:
# 0.77
rf.fit(train_X , train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': name, 'poi': rf_pred})
sub.to_csv(store_path + 'midterm_rf.csv', index=False)

In [0]:
# 0.82
rf_best_param.fit(train_X , train_Y)
rf_best_param_pred = rf_best_param.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': name, 'poi': rf_best_param_pred})
sub.to_csv(store_path + 'midterm_rf_best_param.csv', index=False)